<img src="https://github.com/alan-barzilay/NLPortugues/blob/master/imagens/logo_nlportugues.png?raw=true"  style="height:65%" align="right">


# Lista 6 - LSTM&GRU 
**Nome:** Wesley Seidel Carvalho

**Numero Usp:** 6544342

______________




O objetivo desta lista é fazer com que vocês treinem um modelo de análise de sentimentos utilizando GRU's e LSTM's. Essa lista é semelhante a lista 03 onde aprendemos a usar embeddings e onde você ja recebeu a arquitetura do seu modelo quase pronta. A diferença é que desta vez você ira construir sozinho sua rede e utilizará as camadas que acabamos de aprender: LSTM e GRU.
 
Essa tambêm será a primeira rede recorrente que montaremos, portanto a tokenização será ligeiramente diferente (por exemplo o padding não é mais necessário.)

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
tf.__version__
# '2.2.0-rc3'

'2.2.0-rc3'

## Importando os dados como um dataframe

Para esta lista nós utilizaremos um recorte de 10 mil linhas do dataset **B2W-Reviews01** que consiste em avaliações de mais de 130k compras online no site Americanas.com e [esta disponivel no github](https://github.com/b2wdigital/b2w-reviews01) sob a licensa CC BY-NC-SA 4.01.

In [3]:
b2wCorpus = pd.read_csv("../data/b2w-10k.csv")
b2wCorpus.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958,F,RJ,NaN,NaN,NaN,NaN,NaN
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996,M,SC,NaN,NaN,NaN,NaN,NaN
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984,M,SP,NaN,NaN,NaN,NaN,NaN
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985,F,SP,NaN,NaN,NaN,NaN,NaN
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994,M,MG,NaN,NaN,NaN,NaN,NaN


In [4]:
b2wCorpus["review_text"]

0       Estou contente com a compra entrega rápida o ú...
1       Por apenas R$1994.20,eu consegui comprar esse ...
2       SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...
3       MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...
4       A entrega foi no prazo, as americanas estão de...
                              ...                        
9994    Celular muito rápido, com processador e armaze...
9995    achei o produto muito frágil, o material veio ...
9996    Uma porcaria pois ñ recebi ñ recomendo pra nin...
9997    Maquina excelente,super pratica. recomendo.ent...
9998    Agradeço pelo compromisso, obrigado. ,...........
Name: review_text, Length: 9999, dtype: object


## Pré-processamento 
# <font color='blue'>Questão 1 </font>
Copie suas etapas de préprocessamento da lista 03, ou seja, selecione apenas as colunas relevantes ("review_text" e "recommend_to_a_friend"), converta a coluna "review_text" de uma coluna de `str` para uma coluna de `int` e separe os dados em teste e treino.


In [5]:
# Seu código aqui
from sklearn.model_selection import train_test_split

df = b2wCorpus[['review_text','recommend_to_a_friend']]
df = df.assign(recommend_to_a_friend_new=0)
df['recommend_to_a_friend_new'] = df.recommend_to_a_friend.map({'Yes': 1, 'No': 0})

X_train, X_test, y_train, y_test = train_test_split(df['review_text'], df['recommend_to_a_friend_new'], test_size=0.20, random_state=17)

# df.groupby('recommend_to_a_friend').count()

## Tokenizando




# <font color='blue'>Questão 2 </font>
Utilizando a camada [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) tokenize os inputs.
Declare a camada e então chame a função `adapt()` no seu conjunto de treino para adequar o seu vocabulário aos reviews. 

Note que o uso de padding não é mais necessario.

In [6]:
# Seu código aqui
# Estou inicializando algumas variaveis aqui que serão utilizadas nas demais questões.

VOCAB_SIZE = 5000
MAX_SENTENCE_LENGTH = 100
EMBEDDING_LENGTH = 128
QNT_EPOCAS_TREINO = 50

vectorize_layer = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_sequence_length=MAX_SENTENCE_LENGTH,  # Only valid in INT mode.
)

vectorize_layer.adapt(X_train.values)

## LSTM&GRU

Agora vamos juntar a camada do tokenizador a nossa camada [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) e definir o resto de nosso modelo.

#  <font color='blue'>Questão 3 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas  [LSTM](https://keras.io/api/layers/recurrent_layers/lstm/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 
 
b) Como foi a performance desta rede em comparação a da lista 3?




**<font color='red'> Sua resposta aqui </font>**

In [7]:
# Seu código aqui

from tensorflow.keras import layers
import random

model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(VOCAB_SIZE+1, output_dim=EMBEDDING_LENGTH, input_length=MAX_SENTENCE_LENGTH),
    tf.keras.layers.LSTM(
        units=EMBEDDING_LENGTH, 
        activation="tanh",
        recurrent_activation="sigmoid",
    ),

#     layers.Dense(128, activation='relu'),
    layers.Dropout(0.4, seed=random.randint(0,15)),
    
    layers.Dense(units=1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])

model.fit(X_train, y_train,epochs=QNT_EPOCAS_TREINO)

model.summary()

loss, accuracy = model.evaluate(x=X_test,y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/50
250/250 [==============================] - 62s 248ms/step - loss: 0.5677 - acc: 0.7483
Epoch 2/50
250/250 [==============================] - 65s 261ms/step - loss: 0.5488 - acc: 0.7431
Epoch 3/50
250/250 [==============================] - 68s 274ms/step - loss: 0.5481 - acc: 0.7486
Epoch 4/50
250/250 [==============================] - 33s 132ms/step - loss: 0.5591 - acc: 0.7547
Epoch 5/50
250/250 [==============================] - 32s 128ms/step - loss: 0.5551 - acc: 0.7563
Epoch 6/50
250/250 [==============================] - 29s 118ms/step - loss: 0.5474 - acc: 0.7538
Epoch 7/50
250/250 [==============================] - 33s 131ms/step - loss: 0.5014 - acc: 0.7807
Epoch 8/50
250/250 [==============================] - 33s 130ms/step - loss: 0.4044 - acc: 0.8472
Epoch 9/50
250/250 [==============================] - 32s 128ms/step - loss: 0.3179 - acc: 0.8795
Epoch 10/50
250/250 [==============================] - 37s 146ms/step - loss: 0.2478 - acc: 0.9146
Epoch 11/50
250/250


# b)
Usando as mesmas seguintes configurações para os exercicios desta lista e da lista 3:
- VOCAB_SIZE = 5000
- MAX_SENTENCE_LENGTH = 75
- EMBEDDING_LENGTH = 128
- QNT_EPOCAS_TREINO = 15


Enquanto na lista 3 tivemos os seguintes valores:
- Loss:  1.1264379024505615
- Accuracy:  0.8845000267028809


E agora usando o LSTM conseguimos:
- Loss:  0.31173521280288696
- Accuracy:  0.8794999718666077

Apesar da Acurácia se apresentar menor com o LSTM, o LOSS foi bem menor.

#  <font color='blue'>Questão 4 </font>

a) Defina, compile, treine e avalie seu modelo, utilize camadas [GRU](https://keras.io/api/layers/recurrent_layers/gru/).
Atenção a dimensão do input da camada de embedding, lembre se que < OOV > e < PAD > possuem seus próprios tokens.
 
 
 
b) Como foi a performance desta rede em comparação a da lista 3?


**<font color='red'> Sua resposta aqui </font>**

In [8]:
# Seu código aqui
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(VOCAB_SIZE+1, output_dim=EMBEDDING_LENGTH, input_length=MAX_SENTENCE_LENGTH),
    tf.keras.layers.GRU(
        units=EMBEDDING_LENGTH, 
        activation="tanh",
        recurrent_activation="sigmoid",
    ),
    layers.Dropout(0.4, seed=random.randint(0,15)),
    layers.Dense(units=1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])

model.fit(X_train, y_train,epochs=QNT_EPOCAS_TREINO)

model.summary()

loss, accuracy = model.evaluate(x=X_test,y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/50
250/250 [==============================] - 21s 85ms/step - loss: 0.5683 - acc: 0.7461
Epoch 2/50
250/250 [==============================] - 21s 85ms/step - loss: 0.5651 - acc: 0.7530
Epoch 3/50
250/250 [==============================] - 24s 96ms/step - loss: 0.5517 - acc: 0.7551
Epoch 4/50
250/250 [==============================] - 26s 105ms/step - loss: 0.2751 - acc: 0.8947
Epoch 5/50
250/250 [==============================] - 22s 88ms/step - loss: 0.1726 - acc: 0.9422
Epoch 6/50
250/250 [==============================] - 23s 92ms/step - loss: 0.1240 - acc: 0.9626
Epoch 7/50
250/250 [==============================] - 23s 92ms/step - loss: 0.0879 - acc: 0.9765
Epoch 8/50
250/250 [==============================] - 21s 86ms/step - loss: 0.0649 - acc: 0.9830
Epoch 9/50
250/250 [==============================] - 21s 85ms/step - loss: 0.0521 - acc: 0.9870
Epoch 10/50
250/250 [==============================] - 21s 85ms/step - loss: 0.0469 - acc: 0.9881
Epoch 11/50
250/250 [=======


# b)
Usando as mesmas seguintes configurações para os exercicios desta lista e da lista 3:
- VOCAB_SIZE = 5000
- MAX_SENTENCE_LENGTH = 75
- EMBEDDING_LENGTH = 128
- QNT_EPOCAS_TREINO = 15


Enquanto na lista 3 tivemos os seguintes valores:
- Loss:  1.1264379024505615
- Accuracy:  0.8845000267028809


E agora usando o GRU conseguimos:
- Loss:  0.6627704501152039
- Accuracy:  0.8634999990463257

Apesar da Acurácia se apresentar menor com o GRU, o LOSS foi menor, mas não foi menor do que o LSTM.

## Redes Bi-direcionais
#  <font color='blue'>Questão 5 </font>

a) Defina, compile, treine e avalie um novo modelo que utilize contexto em ambas as direções usando a camada [`Bidirectional()`](https://keras.io/api/layers/recurrent_layers/bidirectional/), seja com camadas GRU ou LSTM.


b) Como foi sua performance em relação as questões anteriores com contexto unidirecional?

**<font color='red'> Sua resposta aqui </font>**

In [9]:
# Seu código aqui
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding

from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.Input(shape=(1,), dtype=tf.string),
    vectorize_layer,
    tf.keras.layers.Embedding(VOCAB_SIZE+1, output_dim=EMBEDDING_LENGTH, input_length=MAX_SENTENCE_LENGTH),

    # Camada Bidirecional
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(units=EMBEDDING_LENGTH, activation="tanh", recurrent_activation="sigmoid"),
        backward_layer=tf.keras.layers.LSTM(units=EMBEDDING_LENGTH, activation="relu", recurrent_activation="sigmoid", go_backwards=True)
    ),   
    
    layers.Dropout(0.4, seed=random.randint(0,15)),
    layers.Dense(units=1, activation='sigmoid'),
])
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])

model.fit(X_train, y_train,epochs=QNT_EPOCAS_TREINO)

model.summary()

loss, accuracy = model.evaluate(x=X_test,y=y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

Epoch 1/50
250/250 [==============================] - 53s 211ms/step - loss: 3475491520512.0000 - acc: 0.7732
Epoch 2/50
250/250 [==============================] - 45s 181ms/step - loss: 0.4291 - acc: 0.7966
Epoch 3/50
250/250 [==============================] - 47s 188ms/step - loss: 83.0356 - acc: 0.8357
Epoch 4/50
250/250 [==============================] - 43s 173ms/step - loss: 705388.2500 - acc: 0.8592
Epoch 5/50
250/250 [==============================] - 40s 160ms/step - loss: 153768.4688 - acc: 0.8952
Epoch 6/50
250/250 [==============================] - 41s 162ms/step - loss: 1657.9614 - acc: 0.9095
Epoch 7/50
250/250 [==============================] - 40s 162ms/step - loss: 1687.5179 - acc: 0.9042
Epoch 8/50
250/250 [==============================] - 40s 161ms/step - loss: 481.9288 - acc: 0.8942
Epoch 9/50
250/250 [==============================] - 40s 161ms/step - loss: 1159.1874 - acc: 0.9041
Epoch 10/50
250/250 [==============================] - 41s 164ms/step - loss: 133.81

# b) 
Usando as mesmas seguintes configurações para os exercicios desta lista e da lista 3:
- VOCAB_SIZE = 5000
- MAX_SENTENCE_LENGTH = 75
- EMBEDDING_LENGTH = 128
- QNT_EPOCAS_TREINO = 15


Enquanto na lista 3 tivemos os seguintes valores:
- Loss:  1.1264379024505615
- Accuracy:  0.8845000267028809


E agora usando o GRU conseguimos:
- Loss:  0.6583318114280701
- Accuracy:  0.8669999837875366

Neste exemplo a Acurácia também se apresentou menor com relação ao exercicio da lista 3.